### 1.0 Prepare songs for analysis
This notebook does the following:
 - Segments raw recordings into manually defined bouts
 - Applies a band-pass filter to reject frequencies outside the range

### Define previous steps: 
 - Copy full lenght recordings
 - Segment song bouts with AviaNZ
 - Segment into syllables with chipper

In [2]:
# Reload modules automatically
# to update edited src code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import src.read.paths as pth
from src.audio import segmentation

In [11]:
filelist = list(pth.DATA_PATH.glob('raw/*/*.wav')) # modify to include real path to raw files. Eg: 'wav_list = list(DSLOC.glob('Bird*/Wave/*.wav'))'
len(filelist), np.sort(filelist)


(36,
 array([PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-01-25_000.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-01-25_001.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-01-25_002.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-01-25_003.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-02-02_000.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-02-02_001.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-02-02_002.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-02-12_000.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-02-12_001.wav'),
        PosixPath('/home/nilomr/projects/0.0_great-tit-song/data/raw/00_bagley/2020-02

In [25]:
test_audio = pydub.AudioSegment.from_wav("/home/nilomr/Music/trimmed.wav")
test_audio_bandpass = band_pass_filter(test_audio, 2500, 9000, order = 12)
test_audio_bandpass.export("/home/nilomr/Music/bandpassed.wav", format = "wav")

PosixPath('/home/nilomr/projects/00_gtit')

In [24]:
print(shite)

shite2!
